In [ ]:
# Transformer 다국어 기계 번역 모델 분류기 및 다국어 기계 번역 
# - Hugging Face 라이브러리 적용
# - 입력된 문장을 다국어 기계 번역 모델 분류기을 통한 학술논문(0)/공시정보(1)/뉴스(2)/규정(3)/보고서(4) 분류
# - 입력 문장의 언어 분류 -> 입력 문장에 해당하는 기계 번역 모델 분류 -> 입력 문장에 해당하는 기계 번역
# 1. 학습 목표
# - 구조 최적화 및 파이프라인 단순화
# - AI HUB 금융 학술논문/공시정보/뉴스/규정/보고서 다국어 번역 데이터셋 학습 보강
# - 토크나이징 및 토크나이징 전처리
# - LoRA 적용된 모델 불러오기, 베이스모델/LoRA모델/토크나이저

In [1]:
import torch
import tqdm as notebook_tqdm
import numpy as np
import glob, json, re, os, random, csv, zipfile

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(torch.__version__, device)

print("CUDA 사용 가능 여부:", torch.cuda.is_available())
print("PyTorch CUDA 버전:", torch.version.cuda)
print("빌드 정보:", torch.__version__)
if torch.cuda.is_available():
    print("사용 중인 GPU:", torch.cuda.get_device_name(0))

2.8.0+cu129 cuda
CUDA 사용 가능 여부: True
PyTorch CUDA 버전: 12.9
빌드 정보: 2.8.0+cu129
사용 중인 GPU: NVIDIA GeForce RTX 3060 Laptop GPU


In [19]:
# 라이브러리
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification, pipeline

# - Hugging Face의 pipeline("translation", ...)은 단순 PeftModel 객체를 지원 않는다
# - 대신 PeftModelForSeq2SeqLM 클래스를 사용해야 한다
# - 즉, LoRA를 적용한 Seq2Seq 모델을 불러올 때는 PeftModelForSeq2SeqLM.from_pretrained()을 써야 한다
from peft import PeftModel, PeftModelForSeq2SeqLM
from langdetect import detect

In [7]:
# 분류기 모델 로드
classifier_tokenizer = AutoTokenizer.from_pretrained('./llm_models/translation_model_finance_classification')
classifier_model = AutoModelForSequenceClassification.from_pretrained('./llm_models/translation_model_finance_classification')
classifier = pipeline('text-classification', model=classifier_model, tokenizer=classifier_tokenizer)

The tokenizer you are loading from './llm_models/translation_model_finance_classification' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
Device set to use cuda:0


In [20]:
# 번역 모델(LoRA 적용 M2M100) 로드
base_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/m2m100_418M")

# 각 라벨별로 LoRA weight를 적용한 모델을 불러와서 딕셔너리에 저장
# - 분류 결과가 label=2라면 translation_models[2]를 선택해서 번역 수행
translation_models = {
    0: (
        PeftModelForSeq2SeqLM.from_pretrained(base_model, './llm_models/translation_model_ai_hub_article_lora'),     # 금융 학술 논문(0)
        AutoTokenizer.from_pretrained('./llm_models/translation_model_ai_hub_article_lora')
    ),
    1: (
        PeftModelForSeq2SeqLM.from_pretrained(base_model, './llm_models/translation_model_ai_hub_disclosure'),       # 금융 공시 정보(1)
        AutoTokenizer.from_pretrained('./llm_models/translation_model_ai_hub_disclosure')
    ),
    2: (
        PeftModelForSeq2SeqLM.from_pretrained(base_model, './llm_models/translation_model_ai_hub_news_lora'),        # 금융 뉴스(2)
        AutoTokenizer.from_pretrained('./llm_models/translation_model_ai_hub_news_lora')
    ),
    3: (
        PeftModelForSeq2SeqLM.from_pretrained(base_model, './llm_models/translation_model_ai_hub_regulation_lora'),  # 금융 규정(3)
        AutoTokenizer.from_pretrained('./llm_models/translation_model_ai_hub_regulation_lora')
    ),
    4: (
        PeftModelForSeq2SeqLM.from_pretrained(base_model, './llm_models/translation_model_ai_hub_report_lora'),      # 금융 보고서(4)
        AutoTokenizer.from_pretrained('./llm_models/translation_model_ai_hub_report_lora')
    )
}

d:\AI\Pytorch\deep-learning\.venv\Lib\site-packages\peft\tuners\tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [24]:
# 추론 파이프라인
def inference_pipeline(text):
    # 분류
    row_label = classifier(text)[0]['label']
    # print('row_label : ', row_label)
    label = int(row_label.replace('LABEL_', "")) # LABEL_0 -> 0

    # 해당 라벨의 번역 모델 + 토크나이저 선택
    model, tokenizer = translation_models[label]
    translator = pipeline('translation', model=model, tokenizer=tokenizer)

    # 언어 판별
    lang = detect(text)

    # 번역 수행
    if lang == 'en':
        translated = translator(text, src_lang='en', tgt_lang='ko')[0]['translation_text']    
    elif lang == 'ko':
        translated = translator(text, src_lang='ko', tgt_lang='en')[0]['translation_text']
    else:
        translated = text
    return {'label': label, 'translated_text': translated}

In [25]:
# 추론
text_en = "This research proposes a Transformer model."
result_en = inference_pipeline(text=text_en)
print(result_en)

text_ko = "이 연구는 Transformer 기반 모델을 제안한다."
result_ko = inference_pipeline(text=text_ko)
print(result_ko)

Device set to use cuda:0
Device set to use cuda:0


{'label': 0, 'translated_text': '이 연구는 변압기 모델을 제안한다.'}
{'label': 0, 'translated_text': 'This study proposes a transformer-based model.'}


In [26]:
# 학술논문 (0)
text_article = "This paper introduces a novel deep learning architecture for financial forecasting."
result_article = inference_pipeline(text_article)
print("학술논문(0):", result_article)

# 공시정보 (1)
text_disclosure = "삼성전자는 2025년 4분기 실적을 공시하였다."
result_disclosure = inference_pipeline(text_disclosure)
print("공시정보(1):", result_disclosure)

# 뉴스 (2)
text_news = "Global stock markets rose today as investors reacted positively to the new policy."
result_news = inference_pipeline(text_news)
print("뉴스(2):", result_news)

# 규정 (3)
text_regulation = "금융기관은 자본적정성 규정을 준수해야 한다."
result_regulation = inference_pipeline(text_regulation)
print("규정(3):", result_regulation)

# 보고서 (4)
text_report = "The annual financial report highlights key trends in the banking sector."
result_report = inference_pipeline(text_report)
print("보고서(4):", result_report)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Device set to use cuda:0
Device set to use cuda:0


학술논문(0): {'label': 0, 'translated_text': '이 논문에서는 금융 예측을 위한 새로운 깊은 학습 아키텍처를 소개한다.'}


Device set to use cuda:0


공시정보(1): {'label': 2, 'translated_text': 'Samsung Electronics announced its fourth quarter results for 2025.'}


Device set to use cuda:0


뉴스(2): {'label': 4, 'translated_text': '글로벌 주식시장은 투자자들이 새로운 정책에 긍정적으로 반응했을 때 오늘 상승했다.'}


Device set to use cuda:0


규정(3): {'label': 4, 'translated_text': 'Financial institutions must comply with the regulations on capital stability.'}
보고서(4): {'label': 4, 'translated_text': '연례 재무 보고서에서는 은행계의 핵심 추세를 강조한다.'}
